# Internal notes

- Code has not been tested

# Deploy a model as an online endpoint

Learn to deploy a model, using Azure Machine Learning (AzureML) Python SDK v2. This tutorial will present an overview of how AzureML handles deployment and walk you through the steps to deploy your model to an online endpoint. 

To set the stage for deployment, you'll first configure a workspace, compute resources, and a job environment necessary for deployment. Then you'll 

You'll learn to register a model, create an endpoint and create a first deployment. Then, you'll test the deployment with data and retrieve the deployment details. You'll also create a second deployment and allocate some percentage of production traffic to it. After you obtain logs from the second deployment and are satisfied with its performance, you'll eventually send all the production traffic to your second deployment and delete the first one.

The steps you'll take are:

> * [DELETE] Connect to your AzureML workspace
> * [DELETE] Create your compute resource and job environment
> * [DELETE] Configure authorization
> * Ensure your model is registered
> * Configure environments
> * Import libraries
> * Verify environment
> * Create an endpoint and a first deployment
> * Deploy a trial run
> * Manually send test data to the deployment
> * Get details of the deployment
> * Create a second deployment
> * Manually scale the second deployment
> * Update allocation of production traffic between both deployments
> * Get details of the second deployment
> * Roll out the new deployment and delete the first deployment

## Overview
You can deploy your machine learning model as an [`online endpoint`](https://docs.microsoft.com/azure/machine-learning/concept-endpoints), that is, as a web service in the Azure cloud, so that so you don't have to create and manage the underlying infrastructure. For this deployment, you typically need:

* The model assets (file and metadata) that you want to deploy.
* A scoring script, that is, some code to run as a service. The code executes the model on a given input request. This entry script receives data submitted to a deployed web service and passes it to the model, then returns the model's response to the client. The script is specific to your model. The entry script must understand the data that the model expects and returns. With an MLflow model, as in this tutorial, this script is automatically created for you. Samples of scoring scripts can be found [here](https://github.com/Azure/azureml-examples/tree/sdk-preview/sdk/endpoints/online).

## Prerequisites
If you already completed the earlier Day 1 tutorials, _train model responsibly_ or _create reusable pipeline_, you have everything you need to continue with the tutorial.

If you’ve not completed the earlier tutorials, be sure to do the following: 

- Access an Azure account with an active subscription. If you don't have an Azure subscription, [create a free account](https://azure.microsoft.com/free/) to begin.
- Create an AzureML workspace and a compute instance if you don't have them already. The [Quickstart: Create workspace resources](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources) provides steps that you can follow. Be sure to have enough quota for the compute resources.
    - <mark>**(TODO: specify what "enough quota" means)**</mark>
- Configure the environment: 
    - <mark>#TODO: We should specify what additional environment configuration is needed for inferencing. Dev environment would be set up in article 2.)** ~~Skip if you have already done it. If you haven’t done it, go read tutorial 2. ~~</mark>
- Retrieve the metadata and files for the model you'll deploy 
    - <mark>**(TODO: add links to the files here)**</mark>
    - <mark> MOVE FROM HERE: You'll be working with an MLflow model. AzureML supports no-code deployment of models created and logged with MLflow. This means that you don't have to provide a scoring script or an environment. Note that if you were using a custom model, you'd have to specify the environment and scoring script. See [Customizing MLflow model deployments with scoring script](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?tabs=cli#customizing-mlflow-model-deployments) to learn how to use your scoring script. **(TODO: put a link to how this can be done)**</mark>
> [!IMPORTANT]
> If you typically deploying models using scoring scripts and custom environments and want to achieve the same functionality using MLflow models, we recommend reading [Using MLflow models for no-code deployment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-log-mlflow-models?tabs=wrapper).


## Connect to the workspace
Before you dive in the code, you'll need to connect to your AzureML workspace. The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section, we'll connect to the workspace in which you'll perform deployment tasks.

We're using `DefaultAzureCredential` to get access to the workspace. 
`DefaultAzureCredential` is used to handle most Azure SDK authentication scenarios. 

If `DefaultAzureCredential` doesn't work for you, you can access other available credentials by checking these references: [configure credential example](../../configuration.ipynb) and [azure-identity reference doc](https://docs.microsoft.com/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

However, if you want to use a browser to login and authenticate, you can uncomment the following code to use the `InteractiveBrowserCredential`.

In [1]:
# Handle to the workspace
# from azure.ai.ml import MLClient

# Authentication package
# from azure.identity import InteractiveBrowserCredential
# credential = InteractiveBrowserCredential()

To connect to a workspace, you need identifier parameters—a Subscription ID, Resource Group name and Workspace name. You'll use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required AzureML workspace. This example uses the [default Azure authentication](https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python).

In the next cell, replace the placeholder text `<RESOURCE_GROUP>` with your resource group name, `<SUBSCRIPTION_ID>` with your subscription ID, and `<AZUREML_WORKSPACE_NAME>` with your workspace name. To find these values:

1. In the top right of the Azure Machine Learning studio toolbar, select your workspace name.
2. Copy the value for workspace, resource group and subscription ID into the code.
3. You'll need to copy one value at a time, close the area, paste, then come back for the next one.

In [ ]:
# enter details of your AzureML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AZUREML_WORKSPACE_NAME>"

In [ ]:
# Get a handle to the workspace
ml_client = MLClient(
     DefaultAzureCredential(), subscription_id, resource_group, workspace
)

The result is a handle to the workspace that you'll use to manage other resources and jobs.

> [!IMPORTANT]
> Creating `MLClient` will not connect to the workspace. The client initialization is lazy and will wait for the first time it needs to make a call. In this notebook, this call will happen during compute creation.


## Register the model

If you didn't complete the the earlier Day 1 tutorials, _train model responsibly_ or _create reusable pipeline_, you'll need to register the model. <mark>Deployment of unregistered models is not supported in Azure Machine Learning.</mark> If you already completed either of those tutorials, you can skip to the next section.

In this example, we specify the `path` (where to upload files from) inline. Here, you'll upload the model from your local computer. The SDK automatically uploads the files and registers the model ~~and environment~~. As a best practice for production, you should register the model ~~and environment~~ and specify the registered name and version separately in the code. 

For more information on registering your model as an asset, see [Register your model as an asset in Machine Learning by using the SDK](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-models?tabs=use-local%2Ccli#register-your-model-as-an-asset-in-machine-learning-by-using-the-sdk).

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="mlflow-model/model.pkl",
    type=AssetType.MLFLOW_MODEL,
    name="local-file-example",
    description="Model created from local file.",
)
ml_client.models.create_or_update(file_model)

## Confirm that the model is registered
If you already completed tutorial 4 or 5, you registered an MLflow model as part of the training script. <mark>Deployment of unregistered models is not supported in Azure Machine Learning.</mark> ~~We recommend registering your model as a best practice for production.~~

You can check the **Models** page in AzureML studio to identify the latest version of your registered model. Alternatively, the code below will retrieve the latest version number for you to use.

In [ ]:
registered_model_name = "credit_defaults_model"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

### Key aspects of the model

 <mark> MOVE FROM HERE: You'll be working with an MLflow model. AzureML supports no-code deployment of models created and logged with MLflow. This means that you don't have to provide a scoring script or an environment. Note that if you were using a custom model, you'd have to specify the environment and scoring script. See [Customizing MLflow model deployments with scoring script](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?tabs=cli#customizing-mlflow-model-deployments) to learn how to use your scoring script. **(TODO: put a link to how this can be done)**</mark>
> [!IMPORTANT]
> If you typically deploying models using scoring scripts and custom environments and want to achieve the same functionality using MLflow models, we recommend reading [Using MLflow models for no-code deployment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-log-mlflow-models?tabs=wrapper).

Now that you have a registered model, you can continue to create an endpoint and deployment. In the next section, we'll briefly cover some key details about these topics.

## Endpoints and deployments

After you train a machine learning model, you need to deploy the model so that others can use it to do inferencing. In Azure Machine Learning, you can use **endpoints** and **deployments** to do so.

An **endpoint** is an interface, based on the HTTPS protocol, that clients can call to receive the inferencing (scoring) output of a trained model. It provides: 
- Authentication using "key & token" based auth 
- SSL termination 
- A stable scoring URI (endpoint-name.region.inference.ml.azure.com)


A **deployment** is a set of resources required for hosting the model that does the actual inferencing. 

A single endpoint can contain multiple deployments. Endpoints and deployments are independent Azure Resource Manager resources that appear in the Azure portal.

Azure Machine Learning uses the concept of endpoints and deployments to implement different types of endpoints: [online endpoints](https://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints#what-are-online-endpoints) and [batch endpoints](https://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints#what-are-batch-endpoints). In this tutorial, we'll walk you through the steps of implementing an online endpoint—that is, an endpoint used for receiving data from clients and sending back responses in real-time.

## Create an online endpoint

Now that you have a registered model ~~and an inference script (auto-generated this time)~~, it's time to create your online endpoint. The endpoint name needs to be unique in the entire Azure region. For this tutorial, you'll create a unique name using a universally unique identifier [`UUID`](https://en.wikipedia.org/wiki/Universally_unique_identifier#:~:text=A%20universally%20unique%20identifier%20(UUID,%2C%20for%20practical%20purposes%2C%20unique). For more information on the endpoint naming rules, see [managed online endpoint limits](how-to-manage-quotas.md#azure-machine-learning-managed-online-endpoints).

> [!NOTE]
> Expect the endpoint creation to take approximately 6 to 8 minutes.

> [!TIP]
> * `auth_mode` : Use `key` for key-based authentication. Use `aml_token` for Azure Machine Learning token-based authentication. A `key` doesn't expire, but `aml_token` does expire. For more information on authenticating, see [Authenticate to an online endpoint](https://learn.microsoft.com/azure/machine-learning/how-to-authenticate-online-endpoint).
> * Optionally, you can add a description and tags to your endpoint.

In [ ]:
import uuid

# Create a unique name for the endpoint
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]

We'll create the endpoint using the `ManagedOnlineEndpoint` class.

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# define an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

Using the `MLClient` created earlier, we'll now create the endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [ ]:
# create the online endpoint
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Once you've created an endpoint, you can retrieve it as follows:

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

## Understanding online deployments

The example contains all the files needed to deploy a model to an online endpoint. To deploy a model, you must have:

- Model files (or the name and version of a model that's already registered in your workspace).
- A scoring script, that is, code that takes in user input and returns the scored result.
- An environment in which the model runs. As you'll see, the environment might be a Docker image with Conda dependencies, or it might be a Dockerfile.
- Settings to specify the instance type and scaling capacity

 **Key aspects of a deployment**
    * `name` - Name of the deployment.
    * `endpoint_name` - Name of the endpoint that will contain the deployment.
    * `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
    * `environment` - The environment to use for the deployment. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification. For more information on creating an environment, see 
[Manage Azure Machine Learning environments with the CLI & SDK (v2)](how-to-manage-environments-v2.md#create-an-environment)
    * `code_configuration` - the configuration for the source code and scoring script.
        * `path`- Path to the source code directory for scoring the model.
        * `scoring_script` - Relative path to the scoring file in the source code directory
    * `instance_type` - The VM size to use for the deployment. For the list of supported sizes, see [Managed online endpoints SKU list](reference-managed-online-endpoints-vm-sku-list.md).
    * `instance_count` - The number of instances to use for the deployment.
    
### Deployment using an MLflow model

The registered model is an MLflow model. AzureML supports no-code deployment of models created and logged with MLflow. This means that you don't have to provide a scoring script or an environment during model deployment, as the scoring script and environment are automatically generated for MLflow models. However, if you were using a custom model, you'd have to specify the environment and scoring script. See [Customizing MLflow model deployments with scoring script](https://learn.microsoft.com/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints#customizing-mlflow-model-deployments) to learn how to use a scoring script.
> [!IMPORTANT]
> If you typically deploy models using scoring scripts and custom environments and want to achieve the same functionality using MLflow models, we recommend reading [Using MLflow models for no-code deployment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-log-mlflow-models?tabs=wrapper).

## Deploy the model to the endpoint

You'll begin by creating a single deployment that handles 100% of the incoming traffic. We've chosen an arbitrary color name (*blue*) for the deployment. To create the deployment for our endpoint, we'll use the `ManagedOnlineDeployment` class.

> [!NOTE]
> No need to specify an environment because we are using an MLflow model. MLflow automatically generates the environment.

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment

# Choosing the latest version of our registered model for deployment
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# define an online deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1
)

Using the `MLClient` created earlier, we'll now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
# create the online deployment
blue_deployment = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## Check the status of the endpoint
You can check the status of the endpoint to see whether the model was deployed without error:

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
print(f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}")

In [ ]:
# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## Test the endpoint with sample data

Using the `MLClient` created earlier, we'll get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:

* `endpoint_name` - Name of the endpoint
* `request_file` - File with request data
* `deployment_name` - Name of the specific deployment to test in an endpoint

We'll send a sample request using a [json](https://github.com/Azure/azureml-examples/tree/main/sdk/python/endpoints/online/model-1/sample-request.json) file. <mark>UPDATE Json file and add it to the repo</mark>

In [ ]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="./deploy/sample-request.json",
)

## Get logs of the deployment
Check the logs to see whether the endpoint/deployment were invoked successfuly
If you face errors, see [Troubleshooting online endpoints deployment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints?tabs=cli).

In [ ]:
logs = ml_client.online_deployments.get_logs(name="blue", endpoint_name=online_endpoint_name, lines=50)
print(logs)

## Create a second deployment 
Deploy the model as a second deployment called `green`. For the purpose of our example, you'll deploy the same model. In a real use case, you'd want to deploy a new version of the model or a new model. 

In [ ]:
# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# define an online deployment
green_deployment = ManagedOnlineDeployment(
    name="green",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

# create the online deployment
green_deployment = ml_client.online_deployments.begin_create_or_update(green_deployment).result()

## Scale deployment to handle more traffic

Using the `MLClient` created earlier, we'll get a handle to the `green` deployment. The deployment can be scaled by increasing or decreasing the `instance_count`.

In the following code, you'll increase the VM instance manually. However, note that it is also possible to autoscale online endpoints. Autoscale automatically runs the right amount of resources to handle the load on your application. Managed online endpoints support autoscaling through integration with the Azure monitor autoscale feature. To configure autoscaling, see [autoscale online endpoints](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-autoscale-endpoints?tabs=python).

In [ ]:
# update definition of the deployment
green_deployment.instance_count = 2

# update the deployment
ml_client.online_deployments.begin_create_or_update(endpoint).result()

## Update traffic allocation for deployments
You can split production traffic between deployments. You may first want to test the `green` deployment with sample data, just like you did for the `blue` deployment. Once you've tested your green deployment, allocate a small percentage of traffic to it.

In [ ]:
endpoint.traffic = {"blue": 90, "green": 10}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

You can test traffic allocation by invoking the endpoint several times:

In [ ]:
# You can invoke the endpoint several times
for i in range(20):
    ml_client.online_endpoints.invoke(
        endpoint_name=online_endpoint_name,
        request_file="./deploy/sample-request.json",
    )

Show logs from the `green` deployment to check that there were incoming requests and the model was scored successfully. 

In [ ]:
ml_client.online_deployments.get_logs(name="green", endpoint_name=online_endpoint_name, lines=50)

## Send all traffic to the new deployment
Once you're fully satisfied with your `green` deployment, switch all traffic to it.

In [ ]:
endpoint.traffic = {"blue": 0, "green": 100}
ml_client.begin_create_or_update(endpoint).result()

## Delete the old deployment
Remove the old (blue) deployment:

In [ ]:
ml_client.online_deployments.begin_delete(
    name="blue", endpoint_name=online_endpoint_name
).result()

## Delete the endpoint and deployment

If you aren't going use the deployment, you should delete it with:

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()

## Next Steps
- Mirror traffic
- Batch deployment articles
- autoscaling deployment
- understanding the cost of running endpoints
- getting user logs and metrics

# Sections below could be developed or removed

<mark> @shohei<mark>
### GET DETAILS (@shohei can you add code for these?)
- every detail/status of the endpoint/deployment. This includes checking  
(1) metadata of endpoint/deployment (deployment, container, access, etc), tracking  
(2) operational metrics (operational metrics e.g. latency/throughput/utilization/memory/etc via AZmon) 
(3) system logs (to a level to debug which instance did what) (Log Analytics / Azure Monitor) 
(4) user logs (AppInsight) 

### REQUEST QUOTA  
add reference documentation to MLOps docs: 
- workspace-level quotas
- VMquota
- #cores
- #endpoint quota incraeases (endpoint/deployment)
- #connections

## Check what you need for model deployment
### (*@shohei, we can delete this section, right?, the information is already in the "prerequisites"*)
Again, these resources are needed to be specifed in the definition of Online deployment. 

- Model: 
    - Registered
- Environment (specific to model):
    - conda.yaml and docker image (you don’t need to install docker; you need a link to the docker image)
    - Automatically generated for MLflow models. You don't need to specify this time. If you were using a custom model, you'd have to specify the environment.(TODO: put a link to how this can be done?)
- Compute resource
    - VM SkU is to be specified 
- Scoring script
    - The scoring script which takes input requests and returns the scored results.
    - Automatically generated for MLflow models. You don't need to specify this time.

## Understand the scoring script (DELETE section)

[Customizing MLflow model deployments with scoring script](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?tabs=cli#customizing-mlflow-model-deployments)

Skip explaining scoring script, we can mention we support bringing own scoring script for advanced scenarios and link to [this article](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?tabs=cli#customizing-mlflow-model-deployments) for learners to see how to use a scoring scipt

### TODO:
- Can we show the content of the scoring script (autogenerated by MLflow) and explain it
    - mention that for custom models (not mlflow), you'd need to provide a scoring script and validate it.
    - TODO: Link to an article that shows how to do this


> [!TIP]
> The format of the scoring script for online endpoints is the same format that's used in the preceding version of the CLI and in the Python SDK.

As noted earlier, the script specified in `CodeConfiguration(scoring_script="score.py")` must have an `init()` function and a `run()` function. 


This example uses the [score.py file](https://github.com/Azure/azureml-examples/blob/main/sdk/python/endpoints/online/model-1/onlinescoring/score.py):
__score.py__
:::code language="python" source="~/azureml-examples-main/cli/endpoints/online/model-1/onlinescoring/score.py" :::

The `init()` function is called when the container is initialized or started. Initialization typically occurs shortly after the deployment is created or updated. Write logic here for global initialization operations like caching the model in memory (as we do in this example). The `run()` function is called for every invocation of the endpoint and should do the actual scoring and prediction. In the example, we extract the data from the JSON input, call the scikit-learn model's `predict()` method, and then return the result.